In [2]:
# 一、准备数据

# imdb数据集的目标是根据电影评论的文本内容预测评论的情感标签。

# 训练集有20000条电影评论文本，测试集有5000条电影评论文本，

# 其中正面评论和负面评论都各占一半。 

# 文本数据预处理较为繁琐，

# 包括中文切词（本示例不涉及），

# 构建词典，编码转换，序列填充，构建数据管道等等。 

# 在torch中预处理文本数据一般使用torchtext或者自定义Dataset，

# torchtext功能非常强大，可以构建文本分类，序列标注，问答模型，机器翻译等NLP任务的数据集。

# torchtext常见API一览：

# torchtext.data.Example : 用来表示一个样本，数据和标签 
    
# torchtext.vocab.Vocab: 词汇表，可以导入一些预训练词向量 
    
# torchtext.data.Datasets: 数据集类，__getitem__返回 Example实例, 
    
# torchtext.data.TabularDataset是其子类。 

# torchtext.data.Field : 用来定义字段的处理方法（文本字段，标签字段）创建 Example时的 预处理，batch 时的一些处理操作。
    
# torchtext.data.Iterator: 迭代器，用来生成 batch 
    
# torchtext.datasets: 包含了常见的数据集。
        
import torch
import string, re
import torchtext

# 仅考虑最高频的10000个词
MAX_WORDS = 10000
# 每个样本保留200个词的长度
MAX_LEN = 200
BATCH_SIZE = 20

# 分词方法
tokenizer = lambda x:re.sub('[%s]'%string.punctuation,"",x).split(" ")

# 过滤掉低频词
def filterLowFreqWords(arr, vocabs):
    arr = [[x if x<MAX_WORDS else 0 for x in example]
          for example in arr]
    return arr

# 1. 定义各个字段的预处理方法
TEXT = torchtext.legacy.data.Field(sequential=True, tokenize=tokenizer, lower=True,
                            fix_length=MAX_LEN,postprocessing = filterLowFreqWords)
LABEL = torchtext.legacy.data.Field(sequential=False, use_vocab=False)

# 2.构建表格型dataset
 #torchtext.data.TabularDataset可读取csv,tsv,json等格式
ds_train, ds_test = torchtext.legacy.data.TabularDataset.splits(       
        path='../data/imdb', train='train.tsv',test='test.tsv', format='tsv',        
        fields=[('label', LABEL), ('text', TEXT)],skip_header = False)

# 3. 构建词典
TEXT.build_vocab(ds_train) 

# 4. 构建数据管道迭代器
train_iter, test_iter = torchtext.legacy.data.Iterator.splits(        
    (ds_train, ds_test),  sort_within_batch=True,sort_key=lambda x: len(x.text),        
    batch_sizes=(BATCH_SIZE,BATCH_SIZE))

#查看example信息 
print(ds_train[0].text) 
print(ds_train[0].label)

['it', 'really', 'boggles', 'my', 'mind', 'when', 'someone', 'comes', 'across', 'a', 'movie', 'like', 'this', 'and', 'claims', 'it', 'to', 'be', 'one', 'of', 'the', 'worst', 'slasher', 'films', 'out', 'there', 'this', 'is', 'by', 'far', 'not', 'one', 'of', 'the', 'worst', 'out', 'there', 'still', 'not', 'a', 'good', 'movie', 'but', 'not', 'the', 'worst', 'nonetheless', 'go', 'see', 'something', 'like', 'death', 'nurse', 'or', 'blood', 'lake', 'and', 'then', 'come', 'back', 'to', 'me', 'and', 'tell', 'me', 'if', 'you', 'think', 'the', 'night', 'brings', 'charlie', 'is', 'the', 'worst', 'the', 'film', 'has', 'decent', 'camera', 'work', 'and', 'editing', 'which', 'is', 'way', 'more', 'than', 'i', 'can', 'say', 'for', 'many', 'more', 'extremely', 'obscure', 'slasher', 'filmsbr', 'br', 'the', 'film', 'doesnt', 'deliver', 'on', 'the', 'onscreen', 'deaths', 'theres', 'one', 'death', 'where', 'you', 'see', 'his', 'pruning', 'saw', 'rip', 'into', 'a', 'neck', 'but', 'all', 'other', 'deaths', 'a

In [3]:
print(len(TEXT.vocab))

# index转换成string类型
print(TEXT.vocab.itos[0])
print(TEXT.vocab.itos[1])

# string转换成index
# 未知词
print(TEXT.vocab.stoi['<unk>'])
# padding填充
print(TEXT.vocab.stoi['<pad>'])

# freqs词频
print(TEXT.vocab.freqs['<unk>'])
print(TEXT.vocab.freqs['a'])
print(TEXT.vocab.freqs['good'])


108197
<unk>
<pad>
0
1
0
129453
11457


In [4]:
# 查看数据管道信息
# 注意text第0维度是句子长度

for batch in train_iter:
    features = batch.text
    labels = batch.label
    print(features)
    print(features.shape)
    print(labels)
    break
    

tensor([[ 49,  10,  10,  ...,  11,   4,  82],
        [  7,  42, 317,  ...,   7,   0,  73],
        [116,  26, 116,  ...,  35, 114,  64],
        ...,
        [473,  95,  69,  ...,   1,   1,   1],
        [105,  12,  11,  ...,   1,   1,   1],
        [  9,  70,  18,  ...,   1,   1,   1]])
torch.Size([200, 20])
tensor([1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0])


In [6]:
# 将数据管道组织成torch.utils.data.DataLoader相似的features,label输出形式

class DataLoader:
    def __init__(self,data_iter):
        self.data_iter = data_iter
        self.length = len(data_iter)
    def __len__(self):
        return self.length
    def __iter__(self):
#         此处调整features为batch first
#         并调整label的shape和dtype
        for batch in self.data_iter:
            yield(torch.transpose(batch.text,0,1),
                 torch.unsqueeze(batch.label.float(), dim = 1))
dl_train = DataLoader(train_iter)
dl_test = DataLoader(test_iter)

In [ ]:
# 二、定义模型

# 使用Pytorch通常有三种方式构建模型：
# 使用nn.Sequential按层顺序构建模型，
# 继承nn.Module 基类构建自定义模型，
# 继承nn.Module基类构建模型并辅助应用模型容器 
# (nn.Sequential,nn.ModuleList,nn.ModuleDict)进行封装。

# 由于接下来使用类形式的训练循环，
# 我们将模型封装成torchkeras.Model类来获得类似Keras中高阶模型接口的功能。 
# Model类实际上继承自nn.Module类